[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/monacofj/moeabench/blob/main/examples/example_15.ipynb)

# Example 15: Individual vs Grid-Aggregated Hypervolume Perspectives

This example demonstrates the three scaling modes in MoeaBench controlled by the  parameter:
1. ****: Measures the absolute physical volume dominated in the objective space. This is a direct measure of search progress but depends on objective magnitudes.
2. ****: Normalizes the volume based on the range of all solutions present in the current session (or provided in ). It measures **Aggregated Efficiency** (spanning 0 to 1).
3. ****: Measures **Optimality** by normalizing against the Ground Truth (External Reference). 1.0 represents reaching the theoretical maximum attainable volume.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from MoeaBench import mb
print(f"MoeaBench v{mb.system.version()}")

## 1. Setup the Problem and Calibration

We use **DTLZ2** and calibrate it to enable the **Absolute** mode.

In [ ]:
dtlz2 = mb.mops.DTLZ2(n_var=7, n_obj=3)
dtlz2.calibrate()

print("Running Experiments...")
exp1 = mb.experiment(dtlz2, mb.moeas.NSGA2(population=20, generations=50))
exp1.name = "Baseline NSGA-II"
exp1.run(repeat=5)

exp2 = mb.experiment(dtlz2, mb.moeas.NSGA2(population=100, generations=50))
exp2.name = "Premium NSGA-II"
exp2.run(repeat=5)

In [ ]:
def plot_triple(h1, h2, title_suffix, y_max_raw=None):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))
    v1_raw, v2_raw = h1[0].values[-1,:].mean(), h2[0].values[-1,:].mean()
    v1_rel, v2_rel = h1[1].values[-1,:].mean(), h2[1].values[-1,:].mean()
    v1_abs, v2_abs = h1[2].values[-1,:].mean(), h2[2].values[-1,:].mean()
    y_max = y_max_raw if y_max_raw else max(1.05, v2_raw * 1.05)
    
    def annotate(ax, val1, val2, limit=None, limit_label=None, limit_color='gray'):
        ax.axhline(val1, color='blue', linestyle=':', alpha=0.5)
        ax.text(0.5, val1 - y_max*0.01, f"v1={val1:.3f}", color='blue', fontsize=8, va='top')
        ax.axhline(val2, color='darkorange', linestyle=':', alpha=0.5)
        ax.text(0.5, val2 - y_max*0.01, f"v2={val2:.3f}", color='darkorange', fontsize=8, va='top')
        if limit:
            ax.axhline(limit, color=limit_color, linestyle='--', alpha=0.7, label=limit_label)
            ax.legend(loc='lower right', fontsize=8)
        ax.set_ylim([0, y_max])
        
    mb.view.perf_history(h1[0], h2[0], ax=ax1, title=f"1. Raw\n{title_suffix}")
    annotate(ax1, v1_raw, v2_raw)
    mb.view.perf_history(h1[1], h2[1], ax=ax2, title=f"2. Relative\n{title_suffix}")
    annotate(ax2, v1_rel, v2_rel, limit=1.0, limit_label="Session Max")
    mb.view.perf_history(h1[2], h2[2], ax=ax3, title=f"3. Absolute\n{title_suffix}")
    annotate(ax3, v1_abs, v2_abs, limit=1.0, limit_label="Ground Truth", limit_color='gold')
    plt.tight_layout()
    plt.show()

## 2. Individual Perspective (`joint=False`)

In this mode, each algorithm is evaluated using its own local bounding box (its worst solutions). This creates a "Self-Referenced" view where everyone appears to be making good progress relative to their own failures.

In [ ]:
h1_ind = [
    mb.metrics.hv(exp1, scale='raw', joint=False),
    mb.metrics.hv(exp1, scale='relative', joint=False),
    mb.metrics.hv(exp1, scale='absolute', joint=False)
]
h2_ind = [
    mb.metrics.hv(exp2, scale='raw', joint=False),
    mb.metrics.hv(exp2, scale='relative', joint=False),
    mb.metrics.hv(exp2, scale='absolute', joint=False)
]
plot_triple(h1_ind, h2_ind, "[Individual Perspective]")

## 3. Joint Perspective (`joint=True`)

When we enable `joint=True` (the default), all algorithms share the same global bounding box. The **Baseline** (v1) now has to compete in the same physical universe as the **Premium** (v2) configuration. Observe how the Baseline's volume "shrinks" as the box expands to accommodate the Premium's superior search results.

In [ ]:
ref = [exp1, exp2]
h1_jnt = [
    mb.metrics.hv(exp1, ref=ref, scale='raw', joint=True),
    mb.metrics.hv(exp1, ref=ref, scale='relative', joint=True),
    mb.metrics.hv(exp1, ref=ref, scale='absolute', joint=True)
]
h2_jnt = [
    mb.metrics.hv(exp2, ref=ref, scale='raw', joint=True),
    mb.metrics.hv(exp2, ref=ref, scale='relative', joint=True),
    mb.metrics.hv(exp2, ref=ref, scale='absolute', joint=True)
]
y_max_jnt = h2_ind[0].values[-1,:].mean() * 1.1
plot_triple(h1_jnt, h2_jnt, "[Joint Perspective]", y_max_raw=y_max_jnt)

## Conclusion: The Perspective Paradox

- **Indivividual Mode**: Good for tracking internal convergence (does the algorithm continue to improve?).
- **Joint Mode**: Mandatory for fair scientific comparison. It prevents "Paradoxical Wins" where a poor algorithm looks better simply because its bounding box was smaller.